In [43]:
%pip install openpyxl

In [34]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
from collections import Counter
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp
warnings.filterwarnings("ignore", category=RuntimeWarning)
import os

# Clases

## ACO

In [35]:
import numpy as np
import pandas as pd


class ACOSolver:
    def __init__(self, alpha=1.0, beta=2.0, rho=0.1, max_iters=50, ants=20):
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.max_iters = max_iters
        self.ants = ants

    def sum_cap(self, route):
        """Calcula la capacidad total requerida para una ruta"""
        if not route or len(route) == 0:
            return 0
        return sum(self.demands[node] for node in route if node < len(self.demands))

    def sum_length(self, route):
        """Calcula la longitud total de una ruta incluyendo depot"""
        if not route or len(route) == 0:
            return 0

        total_distance = 0
        # Depot -> primer nodo
        total_distance += self.depot_distances[route[0]]

        # Distancias entre nodos de la ruta
        for i in range(len(route) - 1):
            total_distance += self.distance_matrix[route[i]][route[i + 1]]

        # Último nodo -> depot
        total_distance += self.depot_distances[route[-1]]

        return total_distance

    def solution_length(self, routes):
        """Calcula la longitud total de todas las rutas"""
        if not routes:
            return float('inf')
        return sum(self.sum_length(route) for route in routes if route)

    def solve(self, nodes, distance_matrix, cap):
        """Método principal de resolución"""
        try:
            # Preparar datos
            self._prepare_data(nodes, distance_matrix)

            # Debug: verificar datos
            print(f"Debug: n_nodes={self.n_nodes}, capacity={cap}")
            print(f"Debug: demands shape={len(self.demands)}, depot_distances shape={len(self.depot_distances)}")

            # Resolver con ACO capacitado
            raw_paths = self._capacitated_aco(cap)
            print(f"Debug: raw_paths encontrados: {len(raw_paths) if raw_paths else 0}")

            if not raw_paths:
                print("Warning: No se encontraron rutas en ACO capacitado")
                return []

            # Optimizar cada ruta localmente
            optimized_paths = []
            for i, path in enumerate(raw_paths):
                if path and len(path) > 0:
                    optimized_path = self._local_search_2opt(path)
                    if optimized_path and len(optimized_path) > 0:
                        optimized_paths.append(optimized_path)
                        print(f"Debug: ruta {i} optimizada: {optimized_path}")

            print(f"Debug: {len(optimized_paths)} rutas finales")
            return optimized_paths

        except Exception as e:
            print(f"Error en solve: {str(e)}")
            import traceback
            traceback.print_exc()
            return []

    def _prepare_data(self, nodes, distance_matrix):
        """Prepara los datos para el algoritmo"""
        # Agregar distancias al depot
        nodes = nodes.copy()
        nodes["d0"] = distance_matrix.iloc[0]

        # Extraer datos necesarios
        self.depot_distances = nodes["d0"].values
        self.demands = nodes["demand"].values
        self.distance_matrix = distance_matrix.values
        self.n_nodes = len(nodes)

        print(f"Debug: Datos preparados - nodos: {self.n_nodes}")

    def _capacitated_aco(self, capacity):
        """ACO principal con restricciones de capacidad"""
        try:
            # Preparar matrices
            d = self.distance_matrix.copy()
            np.fill_diagonal(d, np.inf)

            tau = np.ones_like(d) * 0.1  # Feromonas iniciales pequeñas
            eta = np.divide(1.0, d, out=np.zeros_like(d), where=d>0)
            eta[eta == np.inf] = 0

            best_routes = []
            best_distance = float('inf')

            for iteration in range(self.max_iters):
                iteration_routes = []

                for ant in range(self.ants):
                    routes = self._construct_solution(tau, eta, capacity)
                    if routes:
                        iteration_routes.extend(routes)
                        total_distance = self.solution_length(routes)

                        if total_distance < best_distance:
                            best_distance = total_distance
                            best_routes = [route.copy() for route in routes]

                # Actualizar feromonas
                if iteration_routes:
                    self._update_pheromones(tau, iteration_routes)

                if iteration % 10 == 0:
                    print(f"Debug: Iteración {iteration}, mejor distancia: {best_distance:.2f}")

            print(f"Debug: ACO terminado, {len(best_routes)} rutas encontradas")
            return best_routes

        except Exception as e:
            print(f"Error en _capacitated_aco: {str(e)}")
            return []

    def _construct_solution(self, tau, eta, capacity):
        """Construye una solución completa respetando capacidades"""
        try:
            unvisited = set(range(1, self.n_nodes))  # Excluir depot (nodo 0)
            routes = []

            while unvisited:
                route = self._construct_single_route(tau, eta, capacity, unvisited)
                if route:
                    routes.append(route)
                    # Remover nodos visitados
                    for node in route:
                        unvisited.discard(node)
                else:
                    # Si no se puede construir más rutas, forzar asignación de nodos restantes
                    if unvisited:
                        remaining = list(unvisited)
                        # Crear rutas individuales para nodos que no caben
                        for node in remaining:
                            if self.demands[node] <= capacity:
                                routes.append([node])
                                unvisited.remove(node)
                    break

            return routes

        except Exception as e:
            print(f"Error en _construct_solution: {str(e)}")
            return []

    def _construct_single_route(self, tau, eta, capacity, available_nodes):
        """Construye una sola ruta respetando capacidad"""
        try:
            route = []
            current_capacity = capacity
            remaining_nodes = available_nodes.copy()
            current_node = 0  # Empezar desde depot

            while remaining_nodes:
                # Filtrar nodos que caben en la capacidad restante
                feasible_nodes = [node for node in remaining_nodes
                                if self.demands[node] <= current_capacity]

                if not feasible_nodes:
                    break

                # Calcular probabilidades de selección
                probabilities = []
                total_weight = 0

                for node in feasible_nodes:
                    pheromone = tau[current_node, node] if current_node < len(tau) else 0.1
                    heuristic = eta[current_node, node] if current_node < len(eta) else 1.0
                    weight = (pheromone ** self.alpha) * (heuristic ** self.beta)
                    probabilities.append(weight)
                    total_weight += weight

                if total_weight == 0:
                    # Selección aleatoria si no hay información
                    next_node = np.random.choice(feasible_nodes)
                else:
                    # Selección probabilística
                    probabilities = [p / total_weight for p in probabilities]
                    next_node = np.random.choice(feasible_nodes, p=probabilities)

                # Agregar nodo a la ruta
                route.append(next_node)
                remaining_nodes.remove(next_node)
                current_capacity -= self.demands[next_node]
                current_node = next_node

            return route if route else []

        except Exception as e:
            print(f"Error en _construct_single_route: {str(e)}")
            return []

    def _update_pheromones(self, tau, routes):
        """Actualiza feromonas"""
        try:
            # Evaporación
            tau *= (1 - self.rho)

            # Depositar feromonas
            for route in routes:
                if not route or len(route) == 0:
                    continue

                route_distance = self.sum_length(route)
                if route_distance > 0:
                    pheromone_deposit = 1.0 / route_distance

                    # Depot -> primer nodo
                    if route[0] < len(tau):
                        tau[0, route[0]] += pheromone_deposit
                        tau[route[0], 0] += pheromone_deposit

                    # Nodos consecutivos en la ruta
                    for i in range(len(route) - 1):
                        if route[i] < len(tau) and route[i+1] < len(tau):
                            tau[route[i], route[i+1]] += pheromone_deposit
                            tau[route[i+1], route[i]] += pheromone_deposit

                    # Último nodo -> depot
                    if route[-1] < len(tau):
                        tau[route[-1], 0] += pheromone_deposit
                        tau[0, route[-1]] += pheromone_deposit

        except Exception as e:
            print(f"Error en _update_pheromones: {str(e)}")

    def _local_search_2opt(self, route):
        """Optimización local 2-opt simple"""
        if not route or len(route) <= 2:
            return route

        try:
            best_route = route.copy()
            best_distance = self.sum_length(best_route)
            improved = True

            while improved:
                improved = False
                for i in range(1, len(route) - 1):
                    for j in range(i + 1, len(route)):
                        # Crear nueva ruta con 2-opt swap
                        new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
                        new_distance = self.sum_length(new_route)

                        if new_distance < best_distance:
                            best_route = new_route
                            best_distance = new_distance
                            improved = True
                            break
                    if improved:
                        break
                route = best_route

            return best_route
        except:
            return route


def process_single_instance(args):
    """Procesa una instancia individual del dataset"""
    idx, row = args

    try:
        print(f"Procesando instancia {idx}")

        # Cargar datos
        nodes = pd.read_parquet(row['nodes'])
        distance_matrix = pd.read_parquet(row['distance_matrix'])
        vehicle_capacity = row['vehicle_capacity']
        problem_cluster = row['problem_cluster']

        print(f"Instancia {idx}: {len(nodes)} nodos, capacidad {vehicle_capacity}")

        # Configurar solver
        aco_solver = ACOSolver(
            alpha=1.0,
            beta=2.0,
            rho=0.1,
            max_iters=30,  # Reducido para debugging
            ants=min(15, len(nodes))
        )

        # Resolver
        best_routes = aco_solver.solve(nodes, distance_matrix, vehicle_capacity)

        if best_routes and len(best_routes) > 0:
            best_value = aco_solver.solution_length(best_routes)
            n_vehicles = len(best_routes)
            print(f"Instancia {idx} resuelta: {n_vehicles} vehículos, distancia {best_value:.2f}")
        else:
            print(f"Instancia {idx} falló: no se encontraron rutas")
            best_value = None
            n_vehicles = 0

        return {
            'problem_cluster': problem_cluster,
            'vehicle_capacity': vehicle_capacity,
            'aco_best_route': best_routes,
            'aco_best_value': best_value,
            'aco_n_vehicles': n_vehicles,
            'status': 'success' if best_routes else 'no_solution'
        }

    except Exception as e:
        print(f"Error procesando instancia {idx}: {str(e)}")
        import traceback
        traceback.print_exc()

        return {
            'problem_cluster': row.get('problem_cluster', 'unknown'),
            'vehicle_capacity': row.get('vehicle_capacity', 0),
            'aco_best_route': [],
            'aco_best_value': None,
            'aco_n_vehicles': 0,
            'status': f'error: {str(e)}'
        }


def process_dataset(df, n_workers=None):
    """Función principal para procesar el dataset"""
    if n_workers is None:
        n_workers = min(mp.cpu_count(), 4)  # Limitar workers para debugging

    print(f"Procesando {len(df)} instancias con {n_workers} workers")

    results = []

    if n_workers == 1:
        # Procesamiento secuencial para debugging
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            result = process_single_instance((idx, row))
            results.append(result)
    else:
        # Procesamiento paralelo
        with ProcessPoolExecutor(max_workers=n_workers) as executor:
            # Enviar trabajos
            future_to_idx = {
                executor.submit(process_single_instance, (idx, row)): idx
                for idx, row in df.iterrows()
            }

            # Recopilar resultados
            for future in tqdm(as_completed(future_to_idx),
                              total=len(future_to_idx),
                              desc="Procesando"):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    idx = future_to_idx[future]
                    print(f"Error en worker para índice {idx}: {e}")
                    results.append({
                        'problem_cluster': 'error',
                        'vehicle_capacity': 0,
                        'aco_best_route': [],
                        'aco_best_value': None,
                        'aco_n_vehicles': 0,
                        'status': f'worker_error: {str(e)}'
                    })

    return pd.DataFrame(results)


# Función de prueba simple
def test_single_instance():
    """Prueba con una instancia simple para verificar funcionamiento"""
    # Crear datos de prueba simples
    n_nodes = 6
    np.random.seed(42)

    # Crear nodos con demandas
    nodes = pd.DataFrame({
        'demand': [0, 10, 15, 20, 12, 8]  # depot tiene demanda 0
    })

    # Crear matriz de distancias simétrica
    coords = np.random.rand(n_nodes, 2) * 100
    dist_matrix = np.zeros((n_nodes, n_nodes))
    for i in range(n_nodes):
        for j in range(n_nodes):
            dist_matrix[i, j] = np.sqrt(np.sum((coords[i] - coords[j])**2))

    distance_matrix = pd.DataFrame(dist_matrix)
    capacity = 30

    print("Probando instancia simple...")
    solver = ACOSolver(max_iters=20, ants=10)
    routes = solver.solve(nodes, distance_matrix, capacity)

    print(f"Rutas encontradas: {routes}")
    if routes:
        print(f"Distancia total: {solver.solution_length(routes):.2f}")
        print(f"Número de vehículos: {len(routes)}")

    return routes

# Prueba

### Small

In [36]:
C1 = 70
nodo1 = pd.read_parquet('small-10n-c50_70-d10_50_nodes.parquet')
dm1 = pd.read_parquet('small-10n-c50_70-d10_50_dm.parquet')



In [37]:
ACOs1 = ACOSolver()
routes1 = ACOs1.solve(nodo1, dm1, C1)
routes1


Debug: Datos preparados - nodos: 10
Debug: n_nodes=10, capacity=70
Debug: demands shape=10, depot_distances shape=10
Debug: Iteración 0, mejor distancia: 767.27
Debug: Iteración 10, mejor distancia: 651.90
Debug: Iteración 20, mejor distancia: 651.90
Debug: Iteración 30, mejor distancia: 651.90
Debug: Iteración 40, mejor distancia: 651.90
Debug: ACO terminado, 4 rutas encontradas
Debug: raw_paths encontrados: 4
Debug: ruta 0 optimizada: [np.int64(5), np.int64(4), np.int64(2)]
Debug: ruta 1 optimizada: [np.int64(8), np.int64(3)]
Debug: ruta 2 optimizada: [np.int64(1), np.int64(9)]
Debug: ruta 3 optimizada: [np.int64(7), np.int64(6)]
Debug: 4 rutas finales


[[np.int64(5), np.int64(4), np.int64(2)],
 [np.int64(8), np.int64(3)],
 [np.int64(1), np.int64(9)],
 [np.int64(7), np.int64(6)]]

In [38]:
ACOs1.solution_length(routes1)

np.float64(651.9013483720489)

#Resultados


In [39]:

# Cargar el dataset original
df = pd.read_excel('problemset.xlsx')



In [40]:
resultados = process_dataset(df, n_workers=8)


Procesando 78 instancias con 8 workers
Procesando instancia 0Procesando instancia 1Procesando instancia 2Procesando instancia 3Procesando instancia 7


Procesando instancia 5Procesando instancia 6


Error procesando instancia 1: [Errno 2] No such file or directory: 'problemset/in/small-10n-c50_70-d10_50_nodes.parquet'Procesando instancia 4


Error procesando instancia 3: [Errno 2] No such file or directory: 'problemset/in/small-10n-c50_70-d10_50_nodes.parquet'Error procesando instancia 0: [Errno 2] No such file or directory: 'problemset/in/small-10n-c50_70-d10_50_nodes.parquet'Error procesando instancia 7: [Errno 2] No such file or directory: 'problemset/in/small-10n-c50_60-d10_50_nodes.parquet'Error procesando instancia 2: [Errno 2] No such file or directory: 'problemset/in/small-10n-c50_70-d10_50_nodes.parquet'Error procesando instancia 5: [Errno 2] No such file or directory: 'problemset/in/small-10n-c50_60-d10_50_nodes.parquet'Error procesando instancia 6: [Errno 2] No such file or 

Procesando:   0%|          | 0/78 [00:00<?, ?it/s]Traceback (most recent call last):



Error procesando instancia 4: [Errno 2] No such file or directory: 'problemset/in/small-10n-c50_70-d10_50_nodes.parquet'


  File "/tmp/ipython-input-2381656658.py", line 285, in process_single_instance
    nodes = pd.read_parquet(row['nodes'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 667, in read_parquet
    return impl.read(
           ^^^^^^^^^^
  File "/tmp/ipython-input-2381656658.py", line 285, in process_single_instance
    nodes = pd.read_parquet(row['nodes'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-2381656658.py", line 285, in process_single_instance
    nodes = pd.read_parquet(row['nodes'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Traceback (most recent call last):
Traceback (most recent call last):
  File "/tmp/ipython-input-2381656658.py", line 285, in process_single_instance
    nodes = pd.read_parquet(row['nodes'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 667, in read_parquet
    return impl.read(
           ^^^^^^^^^^
  File "/tmp/ipython-input-2381656658.py", line 285, in process_single_instance
    nodes = pd.read_parquet(row['nodes'])

...

  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 140, in _get_path_or_handle
    handles = get_handle(
              ^^^^^^^^^^^




Error procesando instancia 77: [Errno 2] No such file or directory: 'problemset/in/big-250n-c150_300-d15_nodes.parquet'

  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 140, in _get_path_or_handle
    handles = get_handle(
              ^^^^^^^^^^^


  File "/tmp/ipython-input-2381656658.py", line 285, in process_single_instance
    nodes = pd.read_parquet(row['nodes'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/common.py", line 882, in get_handle
    handle = open(handle, ioargs.mode)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
Traceback (most recent call last):


  File "/usr/local/lib/python3.12/dist-packages/pandas/io/common.py", line 882, in get_handle
    handle = open(handle, ioargs.mode)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 667, in read_parquet
    return impl.read(
           ^^^^^^^^^^
Traceback (most recent call last):
FileNotFoundError: [Errno 2] No such file or directory: 'problemset/in/big-250n-c150_300-d15_nodes.parquet'
FileNotFoundError: [Errno 2] No such file or directory: 'problemset/in/big-250n-c150_300-d15_nodes.parquet'
  File "/tmp/ipython-input-2381656658.py", line 285, in process_single_instance
    nodes = pd.read_parquet(row['nodes'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 267, in read
    path_or_handle, handles, filesystem = _get_path_or_handle(
                                          ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-i

In [41]:
resultados

,problem_cluster,vehicle_capacity,aco_best_route,aco_best_value,aco_n_vehicles,status
0,small-10n-c50_70-d10_50,55,[],None,0,error: [Errno 2] No such file or directory: 'p...
1,small-10n-c50_70-d10_50,50,[],None,0,error: [Errno 2] No such file or directory: 'p...
2,small-10n-c50_70-d10_50,65,[],None,0,error: [Errno 2] No such file or directory: 'p...
3,small-10n-c50_60-d10_50,54,[],None,0,error: [Errno 2] No such file or directory: 'p...
4,small-10n-c50_70-d10_50,60,[],None,0,error: [Errno 2] No such file or directory: 'p...
...,...,...,...,...,...,...
73,big-250n-c150_300-d15,222,[],None,0,error: [Errno 2] No such file or directory: 'p...
74,big-250n-c150_300-d15,276,[],None,0,error: [Errno 2] No such file or directory: 'p...
75,big-250n-c150_300-d15,240,[],None,0,error: [Errno 2] No such file or directory: 'p...
76,big-250n-c150_300-d15,258,[],None,0,error: [Errno 2] No such file or directory: 'p...


In [42]:
resultados.to_excel("ACO_CLA.xlsx")